In [18]:
# Importing dependecies:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.preprocessing import text
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, Dense, Input, Flatten, Activation, Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
import seaborn as sns
plt.switch_backend('agg')
%matplotlib inline
from pandas import compat
compat.PY3 = True

# Configuring Notebook environment:
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = (10.0, 7.5)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
df = pd.read_csv('data/strings/df_clean.csv', index_col=0)
df.head()

,title,ingredients,instructions,ingredients_vector,instructions_vector
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,grammie hamblets deviled crab,celery finely chopped green pepper finely chop...,toss ingredients lightly spoon buttered baking...,"['celery', 'finely', 'chopped', 'green', 'pepp...","['toss', 'ingredients', 'lightly', 'spoon', 'b..."
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,infineon raceway baked beans,skirt steak cut inch dicekosher salt fresh cra...,watch make recipe sprinkle steak salt pepper s...,"['skirt', 'steak', 'cut', 'inch', 'dicekosher'...","['watch', 'make', 'recipe', 'sprinkle', 'steak..."
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,southwestern black bean dip,cups dried black beans picked rinsed cups wate...,saucepan let beans soak enough cold water cove...,"['cups', 'dried', 'black', 'beans', 'picked', ...","['saucepan', 'let', 'beans', 'soak', 'enough',..."
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,sour cream noodle bake,ground chuckone tomato sauce saltfreshly groun...,watch make recipe preheat oven degrees f brown...,"['ground', 'chuckone', 'tomato', 'sauce', 'sal...","['watch', 'make', 'recipe', 'preheat', 'oven',..."
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,sushi renovation,rice brown mediumgrain cookedcup quinoacup swe...,special equipment sushi mat cook brown rice qu...,"['rice', 'brown', 'mediumgrain', 'cookedcup', ...","['special', 'equipment', 'sushi', 'mat', 'cook..."


## Word2Vec:

In [ ]:
# model_ingredients = gs.Word2Vec(df['ingredients_vector'], min_count=1, size= 25, workers=3, window=3, sg=1)
# model_instructions = gs.Word2Vec(df['instructions_vector'], min_count=1, size= 25, workers=3, window=3, sg=1)

In [ ]:
# print(model_ingredients)
# data = model_instructions.most_similar('beef')
# print(data)

## Encoding Text:

In [4]:
X = df[['ingredients', 'instructions']]
y = df['title']
print(X[:5], y[:5])

                                                                       ingredients  \
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi  celery finely chopped green pepper finely chop...   
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO  skirt steak cut inch dicekosher salt fresh cra...   
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK  cups dried black beans picked rinsed cups wate...   
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK  ground chuckone tomato sauce saltfreshly groun...   
kRBQSWtqYWqtkb34FGeenBSbC32gIdO  rice brown mediumgrain cookedcup quinoacup swe...   

                                                                      instructions  
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi  toss ingredients lightly spoon buttered baking...  
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO  watch make recipe sprinkle steak salt pepper s...  
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK  saucepan let beans soak enough cold water cove...  
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK  watch make recipe preheat oven degrees f brown...  
kRBQSWtqYWqtkb34FGeenBSbC32gIdO  special equipment sushi m

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [9]:
vocab_size = 1000
tokenize = text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(X_train)

In [10]:
X_train = tokenize.texts_to_matrix(X_train)

In [15]:
encoder = preprocessing.LabelBinarizer()
encoder.fit(X_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [23]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', 
          optimizer='adam', 
          metrics=['accuracy'])
history = model.fit(X_train, y_train, 
                    batch_size=100, 
                    epochs=2, 
                    verbose=1, 
                    validation_split=0.1)

ValueError: Error when checking target: expected activation_9 to have shape (1,) but got array with shape (1000,)